# Teams and Hotels

Playing around with a backtracking algorithm to solve the problem of assigning teams to hotels during an athletics event. 

In [164]:
import pandas
import numpy as np

In [165]:
teams = pandas.read_csv('teams.csv') \
    .assign(total = lambda df: df['single'] + df['double'])

In [172]:
# TODO: use DataFrame?
hotels = [
    {
        'name': 'Hotel A',
        'capacity_single': 80,
        'capacity_double': 50,
    },
    {
        'name': 'Hotel B',
        'capacity_single': 40,
        'capacity_double': 20,
    },
    {
        'name': 'Hotel C',
        'capacity_single': 40,
        'capacity_double': 20,
    },
    {
        'name': 'Hotel D',
        'capacity_single': 80,
        'capacity_double': 55,
    },
    
]


In [173]:
teams.sort_values(['total'], ascending=False, inplace=True)

In [180]:
from itertools import cycle, islice

def solve(hotels, teams):
    for hotel in hotels:
        hotel['remaining_single'] = hotel['capacity_single']
        hotel['remaining_double'] = hotel['capacity_double']

    teams_hotels = teams.assign(hotel=None).reset_index(drop=True)
    
    if not place_team(hotels, teams_hotels, 0):
        return 'No solution exists.'
    else:
        return teams_hotels
    
def cycle_hotels(hotels, i):
    # - First hotel is tried first (should be filled to capacity)
    # - Round-robin through the other hotels
    cycled_hotels = hotels[:1]
    num_cycled_hotels = len(hotels) - 1
    offset = i % num_cycled_hotels
    cycled_hotels += islice(cycle(hotels[1:]), offset, offset + num_cycled_hotels)
    return cycled_hotels
    
def place_team(hotels, teams_hotels, i):
    if i >= len(teams_hotels):
        # Exit condition -- all teams have been placed
        return True
    
    t = teams_hotels.loc[i]

    cycled_hotels = cycle_hotels(hotels, i)

    for hotel in cycled_hotels:
        if can_place_team(hotel, t):
            teams_hotels.loc[i, 'hotel'] = hotel['name']
            hotel['remaining_single'] -= t['single']
            hotel['remaining_double'] -= t['double']
            
            if place_team(hotels, teams_hotels, i + 1):
                return True
            
            # Backtrack
            teams_hotels.loc[i, 'hotel'] = None
            hotel['remaining_single'] += t['single']
            hotel['remaining_double'] += t['double']
            
    return False

def can_place_team(hotel, team):
    return team['hotel'] is None and \
           hotel['remaining_single'] >= team['single'] and \
           hotel['remaining_double'] >= team['double']


In [181]:
solve(hotels, teams)

team       GBR
single      30
double      55
total       85
hotel     None
Name: 0, dtype: object
team       GER
single      50
double      10
total       60
hotel     None
Name: 1, dtype: object
team       HUN
single      10
double      35
total       45
hotel     None
Name: 2, dtype: object
team       SWE
single      20
double       0
total       20
hotel     None
Name: 3, dtype: object
team       NOR
single      10
double      10
total       20
hotel     None
Name: 4, dtype: object
team       FIN
single      10
double       5
total       15
hotel     None
Name: 5, dtype: object
team       ISL
single       5
double       5
total       10
hotel     None
Name: 6, dtype: object


,team,single,double,total,hotel
0,GBR,30,55,85,Hotel D
1,GER,50,10,60,Hotel A
2,HUN,10,35,45,Hotel A
3,SWE,20,0,20,Hotel A
4,NOR,10,10,20,Hotel C
5,FIN,10,5,15,Hotel B
6,ISL,5,5,10,Hotel B


In [176]:
hotels

[{'capacity_double': 50,
  'capacity_single': 80,
  'name': 'Hotel A',
  'remaining_double': 5,
  'remaining_single': 0},
 {'capacity_double': 20,
  'capacity_single': 40,
  'name': 'Hotel B',
  'remaining_double': 10,
  'remaining_single': 25},
 {'capacity_double': 20,
  'capacity_single': 40,
  'name': 'Hotel C',
  'remaining_double': 10,
  'remaining_single': 30},
 {'capacity_double': 55,
  'capacity_single': 80,
  'name': 'Hotel D',
  'remaining_double': 0,
  'remaining_single': 50}]